# 클립 따기

In [1]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def make_clip_video(path,save_path, start_t, end_t):
    clip_video = VideoFileClip(path).subclip((start_t), (end_t))
    clip_video.write_videofile(save_path)

In [10]:
make_clip_video('./아이유.mp4','./output/아이유.mp4','00:04:33.80', '00:04:36.80')

Moviepy - Building video ./output/아이유.mp4.
MoviePy - Writing audio in 아이유TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./output/아이유.mp4



Moviepy - Done !
Moviepy - video ready ./output/아이유.mp4


# 오디오 추출

In [6]:
import moviepy.editor as mp

clip = mp.VideoFileClip('./output/아이유.mp4')
clip.audio.write_audiofile('./아이유.mp3')

MoviePy - Writing audio in ./아이유.mp3


MoviePy - Done.


# 오디오를 array로

### 아이유

In [41]:
import pydub
import numpy as np
import librosa

# Load the audio segment from the MP3 file
audio_segment = pydub.AudioSegment.from_mp3("./iu.mp3")

# Convert the audio segment to a NumPy array
sample_array = np.array(audio_segment.get_array_of_samples())
# Resample the audio array to 16000 Hz using Librosa
target_sr = 16000
# 기존의 sampling rate을 알야아한다
y_resampled = librosa.resample(sample_array.astype(float), orig_sr=87000, target_sr=target_sr)

# Normalize the audio
normalized_audio = librosa.util.normalize(y_resampled)
from IPython.display import Audio
Audio(normalized_audio, rate=16000)


In [30]:
from transformers import SpeechT5ForSpeechToSpeech, SpeechT5Processor
from datasets import load_dataset
import torch
from transformers import SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")

c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at microsoft/speecht5_vc were not used when initializing SpeechT5ForSpeechToSpeech: ['speecht5.encoder.prenet.pos_conv_embed.conv.weight_g', 'speecht5.encoder.prenet.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing SpeechT5ForSpeechToSpeech from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpeechT5ForSpeechToSpeech from the checkpoint of a model that you exp

In [42]:
sampling_rate = 16000
inputs = processor(audio=normalized_audio, sampling_rate = sampling_rate, return_tensors='pt')

In [43]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

### 타짜

In [63]:
import pydub
import numpy as np
import librosa

# Load the audio segment from the MP3 file
tazza = pydub.AudioSegment.from_mp3("./시나리오.mp3")

# Convert the audio segment to a NumPy array
tazza = np.array(tazza.get_array_of_samples())
# Resample the audio array to 16000 Hz using Librosa
target_sr = 16000
# 기존의 sampling rate을 알야아한다
y_resampled_tazza = librosa.resample(tazza.astype(float), orig_sr=87000, target_sr=target_sr)

# Normalize the audio
normalized_audio_tazza = librosa.util.normalize(y_resampled_tazza)
from IPython.display import Audio
Audio(normalized_audio_tazza, rate=16000)

In [97]:
normalized_audio.shape

(49149,)

In [91]:
speaker_embeddings = torch.tensor(normalized_audio_tazza).unsqueeze(0)
speaker_embeddings = speaker_embeddings.to(torch.float32)

import torch.nn as nn

# 선형 레이어를 사용하여 입력 텐서의 차원을 변경
linear_layer = nn.Linear(42012, 512)
input_tensor_reduced = linear_layer(speaker_embeddings)

In [94]:
speech = model.generate_speech(inputs["input_values"], input_tensor_reduced, vocoder=vocoder)

In [95]:
Audio(speech, rate=16000)

In [77]:
speaker_embeddings.shape, inputs['input_values'].shape

(torch.Size([1, 42012]), torch.Size([1, 49149]))

In [80]:
import torch.nn as nn

# 입력 텐서의 형태를 (1, 42012)으로 가정합니다.
input_tensor = torch.randn(1, 42012)

# 선형 레이어를 사용하여 입력 텐서의 차원을 변경
linear_layer = nn.Linear(42012, 512)
input_tensor_reduced = linear_layer(input_tensor)

# 변환된 입력 텐서의 형태 확인
print(input_tensor_reduced.shape)  # (1, 512)


torch.Size([1, 512])


In [96]:
model

SpeechT5ForSpeechToSpeech(
  (speecht5): SpeechT5Model(
    (encoder): SpeechT5EncoderWithSpeechPrenet(
      (prenet): SpeechT5SpeechEncoderPrenet(
        (feature_encoder): SpeechT5FeatureEncoder(
          (conv_layers): ModuleList(
            (0): SpeechT5GroupNormConvLayer(
              (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
              (activation): GELUActivation()
              (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
            )
            (1-4): 4 x SpeechT5NoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
            (5-6): 2 x SpeechT5NoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
          )
        )
        (feature_projection): SpeechT5FeatureProjection(
          (layer_norm): LayerNorm((512,